Here we will be making a plotter for the time series data obtained for:
i) Different Tx powers and ii) Different positions

Firstly importing the required modules

In [ ]:
import plotly.graph_objs as go
import numpy as np
from ipywidgets import widgets
import scipy as sp

Opening the datafile containing the saved parameters

In [ ]:
data_file = open("data/basic_parameters.dat", "rb")
print("Datafile has been opened!")

Loading the saved numpy objects into the memory

In [ ]:
npz_obj = np.load(data_file)

x_axis = npz_obj['x_axis']

x_axis_list = []

for item_x in x_axis:
    
    x_axis_list.append(str(item_x))
    

data_tensor = npz_obj['data_tensor']

mean_mat = npz_obj['mean_mat']

power_list = npz_obj['power_list']

Testing the loaded variables

In [ ]:
power_list = power_list.tolist()

In [ ]:
median_mat = np.median(data_tensor, axis=2)

Closing the datafile

In [ ]:
data_file.close()
print("Datafile has been closed!")

In [ ]:
fig1 = go.Figure()
fig2 = go.Figure()

In [ ]:
default_tx_pwr_ind = 4 
default_pos_ind = 0

textbox_pwr = widgets.Dropdown(
    description='Tx Power:',
    value=power_list[default_tx_pwr_ind],
    options=power_list
)

textbox_pos = widgets.Dropdown(
    description='Position (ft):   ',
    value=x_axis_list[default_pos_ind],
    options=x_axis_list
)

container = widgets.HBox([textbox_pwr, textbox_pos])

In [ ]:
trace1 = go.Scatter(x=np.arange(data_tensor.shape[2]), y=data_tensor[default_tx_pwr_ind, default_pos_ind, :], mode='lines+markers', name='Raw Data')
trace2 = go.Scatter(x=np.arange(data_tensor.shape[2]), y=np.ones(data_tensor.shape[2])*mean_mat[default_tx_pwr_ind, default_pos_ind], mode='lines', name='Average')
trace3 = go.Scatter(x=np.arange(data_tensor.shape[2]), y=np.ones(data_tensor.shape[2])*median_mat[default_tx_pwr_ind, default_pos_ind], mode='lines', name='Median')
trace4 = go.Histogram(x=data_tensor[default_tx_pwr_ind, default_pos_ind, :], histnorm='probability')

In [ ]:
fig1.add_trace(trace1)
fig1.add_trace(trace2)
fig1.add_trace(trace3)

fig2.add_trace(trace4)

fig1.update_layout(
    title={
        'text': "RSSI Time Series",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Time Step",
    yaxis_title="RSSI Power (dBm)")

fig1.update_xaxes(rangeslider_visible=True)

fig2.update_layout(
    title={
        'text': "Normalized Histogram",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="RSSI",
    yaxis_title="Count")

In [ ]:
g1 = go.FigureWidget(fig1, layout=go.Layout(title=dict(text='Time Series for RSSI')))
g2 = go.FigureWidget(fig2)

In [ ]:
def response(change):
    
    pwr_val = textbox_pwr.value
    pos_val = textbox_pos.value
    
    ind_pwr = power_list.index(pwr_val)
    ind_pos = x_axis_list.index(pos_val)
    
    y = data_tensor[ind_pwr, ind_pos, :]
    
    with g1.batch_update():
            g1.data[0].y = y
            g1.data[1].y = np.ones(len(y))*mean_mat[ind_pwr][ind_pos]
            g1.data[2].y = np.ones(len(y))*median_mat[ind_pwr][ind_pos]
    
    with g2.batch_update():
            g2.data[0].x = y
            
            
textbox_pwr.observe(response, names="value")
textbox_pos.observe(response, names="value")



In [ ]:
display(widgets.VBox([container, g1, g2]))